In [1]:
import boto3
import configparser
import json

In [2]:
config = configparser.ConfigParser()
config.read('dwh.cfg')

KEY = config.get("AWS", "KEY")
SECRET = config.get("AWS", "SECRET")

HOST = config.get("CLUSTER", "HOST")
DBNAME = config.get("CLUSTER", "DB_NAME")
USER = config.get("CLUSTER", "DB_USER")
PASSWORD = config.get("CLUSTER", "DB_PASSWORD")
PORT = config.get("CLUSTER", "DB_PORT")

In [4]:
s3 = boto3.resource('s3',
                       region_name="us-west-2",
                       aws_access_key_id=KEY,
                       aws_secret_access_key=SECRET
                   )

In [7]:
bucket = s3.Bucket('s3://udacity-dend/log_json_path.json')

In [3]:
try:
    ec2 = boto3.resource('ec2',
                           region_name="us-west-2",
                           aws_access_key_id=KEY,
                           aws_secret_access_key=SECRET
                        )
    s3 = boto3.resource('s3',
                           region_name="us-west-2",
                           aws_access_key_id=KEY,
                           aws_secret_access_key=SECRET
                       )

    iam = boto3.client('iam',aws_access_key_id=KEY,
                         aws_secret_access_key=SECRET,
                         region_name='us-west-2'
                      )

    redshift = boto3.client('redshift', region_name='us-west-2',
                            aws_access_key_id=KEY,
                            aws_secret_access_key=SECRET
                           )

except Exception as e:
    print(e)

In [6]:
try:
    assume_role_policy_document = json.dumps({
        "Version": "2012-10-17",
        "Statement": [
            {
            "Effect": "Allow",
            "Principal": {
                "Service": "redshift.amazonaws.com"
            },
            "Action": "sts:AssumeRole"
            }
        ]
    })

    dwhRole = iam.create_role(
        Path='/',
        RoleName='dwhRole',
        Description='Programmatic AWS Access on Redshift clusters',
        AssumeRolePolicyDocument=assume_role_policy_document
    )

    iam.attach_role_policy(RoleName='dwhRole',
                           PolicyArn='arn:aws:iam::aws:policy/AmazonS3ReadOnlyAccess'
                          )['ResponseMetadata']['HTTPStatusCode']
    roleArn = iam.get_role(RoleName='dwhRole')['Role']['Arn']

except Exception as e:
    print(e)

An error occurred (InvalidClientTokenId) when calling the CreateRole operation: The security token included in the request is invalid.


In [ ]:
try:
    response = redshift.create_cluster(
        ClusterType='multi-node',
        NodeType='dc2.large',
        NumberOfNodes=4,
        DBNAME=DBNAME,
        ClusterIdentifier='dwhCluster',
        MasterUsername=USER,
        MasterUserPassword=PASSWORD,
        IamRoles=[roleArn]  
    )
except Exception as e:
    print(e)

In [ ]:
def prettyRedshiftProps(props):
    pd.set_option('display.max_colwidth', -1)
    keysToShow = ["ClusterIdentifier", "NodeType", "ClusterStatus", "MasterUsername", "DBName", "Endpoint", "NumberOfNodes", 'VpcId']
    x = [(k, v) for k,v in props.items() if k in keysToShow]
    return pd.DataFrame(data=x, columns=["Key", "Value"])

myClusterProps = redshift.describe_clusters(ClusterIdentifier='dwhCluster')['Clusters'][0]
prettyRedshiftProps(myClusterProps)

In [ ]:
DWH_ENDPOINT = myClusterProps['Endpoint']['Address']
DWH_ROLE_ARN = myClusterProps['IamRoles'][0]['IamRoleArn']
print("DWH_ENDPOINT :: ", endpoint)
print("DWH_ROLE_ARN :: ", roleArn)

In [ ]:
try:
    vpc = ec2.Vpc(id=myClusterProps['VpcId'])
    defaultSg = list(vpc.security_groups.all())[0]
    print(defaultSg)
    defaultSg.authorize_ingress(
        GroupName=defaultSg.group_name,
        CidrIp='0.0.0.0/0',
        IpProtocol='TCP',
        FromPort=int(PORT),
        ToPort=int(PORT)
    )
except Exception as e:
    print(e)

In [ ]:
%load_ext sql

In [ ]:
conn_string="postgresql://{}:{}@{}:{}/{}".format(DWH_DB_USER, DWH_DB_PASSWORD, DWH_ENDPOINT, DWH_PORT,DWH_DB)
print(conn_string)
%sql $conn_string

In [ ]:
# redshift.delete_cluster( ClusterIdentifier=DWH_CLUSTER_IDENTIFIER,  SkipFinalClusterSnapshot=True)